# Agglomerative Clustering

### Background
the Hierarchical Clustering is a tree-based approach of implementation of clustering process. Unlike Divisive Clustering, the Agglomerative Hierarchical Clustering (HAC) using bottom-up approach and in the first step of the algorithm each object is considered as a single cluster. In each step of the algorithm, the most similar two clusters (based on defined distance measure) are combined into a new cluster.

Agglomerative clustering is one of the oldest and most used strategies for handling clustering process. It has many applications including anomaly detection and bioinformatics applications. One of the important tasks of agglomerative clustering is to specify a distance measure between clusters. In this notebook, we used complete linkage strategy for the distance measures. According to the complete linkage strategy, the distance between pair of clusters is defined as the distance between their furthest pair of data objects.  It is important to note that there are wide range of strategies for specifying the distance between two clusters in agglomerative clustering. 

Agglomerative clustering has a very big advantage that it is very ease to do and to understand. In addition, the dendrogram is a common visual tool for showing the hierarchical clustering as a tree diagram. However, not everything is positive with agglomerative clustering. One of the main disadvantages of this algorithm is that it requires a huge storage, a large time complexity, and it rarely provides the best solution. It is very difficult to perform hierarchical clustering on very large datasets because of the time complexity O(kn2) where k is the number of clusters and n is the number of inputs to be clustered. In addition, it is a greedy algorithm and once we created a cluster, we “can’t go back”.  


### Implementation

In this notebook, we have written a general approach to implement Hierarchial Agglomerative Clustering (HAC). 

We have used *Complete Linkeage Strategy* as the distance measure between two clusters. 

### Rules

We have defined a set of assumptions and rules that should be followed:

-> The input is a Python dictionary that maps object ID (integer) to 1D array

-> Distance function should be configurable - by default the distance between pair of clusters is based on the *Complete Linkeage Strategy*

-> The expected result is set of tuples that each one of them includes the IDs of the objects within that cluster

### Getting Started

First, let's import the libraries nesessary for this notebook 

In [733]:
# Import numpy which is the single important library for this algorithm
import numpy as np

### Create Input

In case you already have data, you can load it and skip this part. However, please make sure you follow the required data structure for the next phases.

In [734]:
# for a given values of array size and number of clusters, the following method generates a random input
def create_input(array_size=10, clusters=5):
    np.random.seed(5)
    points = {}
    for i in range(clusters):
        points[i] = np.random.rand(array_size, )
    print('We just initilized {} inputs, each from {} dimension'.format(clusters, array_size))
    return points
points = create_input()

We just initilized 5 inputs, each from 10 dimension


let's examine one of the records

In [735]:
points[0]

array([0.22199317, 0.87073231, 0.20671916, 0.91861091, 0.48841119,
       0.61174386, 0.76590786, 0.51841799, 0.2968005 , 0.18772123])

### Distance Measure

As mentioned above, you need to specify a distance measure for the agglomerative clustering. We decided to use Complete Linkage Strategy.

In [727]:
# Calculate the distance between two arrays
def calculate_distance(p1, p2):
    a1, b1 = min(p1), max(p1)
    a2, b2 = min(p2), max(p2)
    return max(abs(b2-a1), abs(b1-a2))

### Implementation

The following code cells includes two general flow of the algorithm as we defined. Important notes:
- We only need to calculate the distances in the first step of the algorithm. The distance between an "old cluster" and a "new cluster" is basically the maximum distance between the "old cluster" and the two "old clusters" the combines the "new cluster". In other words, we already have that distance and we don't need to calculate it again.
- Throughout the algorithm, we keep two important objects: 
    - clusters: set of tuples that represents the current clusters
    - points_dist: dictionary that stores the distances between each pair of clusters in the current phase
- In each iteration, we update both the list of current clusters and the dictionary of distances between clusters.
- Assuming that the k (number of clusters) is lower than n (the number of observations), we need to go through n-k iterations to achieve k clusters.

In [771]:
def update_points_dist(points_dist,clusters, new_cluster):
    '''  Update of the distance dictionary.
    Note that we delete the "old pairs" that are not important anymore.'''
    
    # Pulling out the clusters that are being merged
    old_cluster_a = new_cluster[0]
    old_cluster_b = new_cluster[1]
    
    # Iterate through the list of current clusters, delete unimportant pairs from the distance dictinoary
    # In addition, find and update the maximum value between each "old cluster" and the "new cluster"
    for c3 in clusters:
        if ((c3),(old_cluster_a)) in points_dist.keys():
            value =  points_dist[(c3),(old_cluster_a)]
            del points_dist[(c3),(old_cluster_a)]
        else:
            value =  points_dist[(old_cluster_a),(c3)]
            del points_dist[(old_cluster_a),(c3)]
        if ((c3),(old_cluster_b)) in points_dist.keys():
            value =  max(value, points_dist[(c3),(old_cluster_b)])
            del points_dist[(c3),(old_cluster_b)]
        else:
            value =  max(value, points_dist[(old_cluster_b),(c3)])
            del points_dist[(old_cluster_b),(c3)]
            
        # Create a new pair in distance dictionary between the "old cluster" and the "new cluster"
        points_dist[old_cluster_a+old_cluster_b, c3] = value
    
    # delete the combintation of the two clusters that represents the "new clusters" since we don't need them anymore
    del points_dist[new_cluster]
    
    # return the updated distance dictionary
    return points_dist

In [777]:
def agglomerative_clustering(points, k=5):
    ''' Perform agglomerative clustering for 
    a given dictionary of points and hyperparameter k (number of clusters). '''
        
    # First step: convert all the clusters into a set of tuples
    clusters = list(zip(points.keys()))
    
    # If the required K is similar or above the number of initial clusters then we done
    if k >= len(clusters):
        return clusters
    
    # Create the dictionary of distances between each pair of clusters. Again, we only need to do that in this phase.
    points_dist = {}
    for c1 in clusters:
        for c2 in clusters:
            if (c1,c2) not in list(points_dist.keys()) and (c2,c1) not in list(points_dist.keys()) and c1!= c2:
                points_dist[(c1, c2)] = calculate_distance(points[c1[0]], points[c2[0]])
                
    # Perform the clustering process until we achieve K clusters (in total n-K iterations)
    # Note that in each iteration, we remove two clusters and add a new cluster
    while k < len(clusters):
        new_cluster = min(points_dist, key=points_dist.get)
        clusters.remove(new_cluster[0])
        clusters.remove(new_cluster[1])
        points_dist = update_points_dist(points_dist, clusters, new_cluster)
        clusters.append(new_cluster[0]+new_cluster[1])
    return clusters

In [778]:
%%time
final_clusters= agglomerative_clustering(points, 2)

Wall time: 0 ns


In [779]:
final_clusters

[(3,), (0, 2, 1, 4)]